### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/106_TROCAS7_Fusion_Apr2021_PEAKS_164/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/106_TROCAS7_Fusion_Apr2021_PEAKS_164


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 106_TROCAS7_Fusion_Apr2021_PEAKS_164_peptide.csv
 106_TROCAS7_Fusion_Apr2021_PEAKS_164_protein.csv
 106_TROCAS7_Fusion_Apr2021_PEAKS_164_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [5]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup106 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/106_TROCAS7_Fusion_Apr2021_PEAKS_164/106_TROCAS7_Fusion_Apr2021_PEAKS_164_peptide.csv")

# remove redundant rows
peaksdb106 = pd.DataFrame.drop_duplicates(peaksdbdup106)

print(peaksdb106.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb106.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb106['#Spec']
del peaksdb106['Accession']
del peaksdb106['PTM']
del peaksdb106['AScore']

mean_length = peaksdb106['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup106))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb106))

#look at the dataframe
peaksdb106.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-106-Chav-DUP', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-106-Chav-DUP', 'Accession', 'PTM', 'AScore'],
      dtype='object')
11.541176470588235
# redundant peaksdb peptides in combined dataframe 85
# nonredundant peaksdb peptides in combined dataframe 85


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,80.22,1881.8350,18,2.3,941.9269,90.07,5150000.0,102,15075,20210408_Trocas7_659_Chav106_DDA_120min_2.raw
1,LGEHNIDVLEGNEQFINAAK,65.43,2210.0967,20,1.3,737.7071,93.19,7910000.0,102,15638,20210408_Trocas7_659_Chav106_DDA_120min_2.raw
2,SSGSSYPSLLQC(+57.02)LK,63.51,1525.7446,14,1.5,763.8807,98.26,1040000.0,102,16521,20210408_Trocas7_659_Chav106_DDA_120min_2.raw
3,IITHPNFNGNTLDNDIM(+15.99)LIK,55.44,2298.1677,20,1.7,767.0645,102.14,98700.0,102,17175,20210408_Trocas7_659_Chav106_DDA_120min_2.raw
4,LGEHNIDVLEGNEQ(+.98)FINAAK,46.98,2211.0808,20,4.3,738.0374,95.20,83300.0,102,15994,20210408_Trocas7_659_Chav106_DDA_120min_2.raw


In [6]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb106['A'] = peaksdb106['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb106['C'] = peaksdb106['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb106['D'] = peaksdb106['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb106['E'] = peaksdb106['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb106['F'] = peaksdb106['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb106['G'] = peaksdb106['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb106['H'] = peaksdb106['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb106 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb106['I'] = peaksdb106['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb106['K'] = peaksdb106['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb106['L'] = peaksdb106['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb106['M'] = peaksdb106['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb106['N'] = peaksdb106['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb106['P'] = peaksdb106['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb106['Q'] = peaksdb106['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb106['R'] = peaksdb106['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb106['S'] = peaksdb106['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb106['T'] = peaksdb106['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb106['V'] = peaksdb106['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb106['W'] = peaksdb106['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb106['Y'] = peaksdb106['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb106['c-carb'] = peaksdb106['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb106['m-oxid'] = peaksdb106['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb106['n-deam'] = peaksdb106['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb106['q-deam'] = peaksdb106['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb106['stripped_peptide'] = peaksdb106['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb106['stripped_length'] = peaksdb106['stripped_peptide'].apply(len)

peaksdb106['NAAF_num.'] = peaksdb106['Area'] / peaksdb106['stripped_length']

# total the number of modifications in sequence
peaksdb106['ptm-total'] = peaksdb106['c-carb'] + peaksdb106['m-oxid'] + peaksdb106['n-deam'] + peaksdb106['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb106['stripped_IL']= peaksdb106['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb106.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/106_CV_T00_GF_PDB/106B_CV_T00_GF_PDB.csv")

# check out the results
peaksdb106.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,80.22,1881.8350,18,2.3,941.9269,90.07,5150000.0,102,15075,...,0,2,0,0,0,SCLISGWGNTK,11,468181.818182,2,SCLLSGWGNTK
1,LGEHNIDVLEGNEQFINAAK,65.43,2210.0967,20,1.3,737.7071,93.19,7910000.0,102,15638,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,395500.000000,0,LGEHNLDVLEGNEQFLNAAK
2,SSGSSYPSLLQC(+57.02)LK,63.51,1525.7446,14,1.5,763.8807,98.26,1040000.0,102,16521,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,74285.714286,1,SSGSSYPSLLQCLK
3,IITHPNFNGNTLDNDIM(+15.99)LIK,55.44,2298.1677,20,1.7,767.0645,102.14,98700.0,102,17175,...,0,0,1,0,0,IITHPNFNGNTLDNDIMLIK,20,4935.000000,1,LLTHPNFNGNTLDNDLMLLK
4,LGEHNIDVLEGNEQ(+.98)FINAAK,46.98,2211.0808,20,4.3,738.0374,95.20,83300.0,102,15994,...,0,0,0,0,1,LGEHNIDVLEGNEQFINAAK,20,4165.000000,1,LGEHNLDVLEGNEQFLNAAK


In [7]:
# keep only stripped peptide I/L and NAAF
db_106 = peaksdb106[['stripped_IL', 'Area', 'NAAF_num.']]

db_106.set_index('stripped_IL')

# write modified dataframe to new txt file
db_106.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/106B_CV_T00_GF_PDB_ILnaafs.csv")

db_106.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,5150000.0,468181.818182
1,LGEHNLDVLEGNEQFLNAAK,7910000.0,395500.000000
2,SSGSSYPSLLQCLK,1040000.0,74285.714286
3,LLTHPNFNGNTLDNDLMLLK,98700.0,4935.000000
4,LGEHNLDVLEGNEQFLNAAK,83300.0,4165.000000


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep106moddup = peaksdb106[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep106dup = peaksdb106[["stripped_peptide"]]

# deduplicate both of these lists
pep106mod = pep106moddup.drop_duplicates()
pep106 = pep106dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 106:", len(pep106moddup))
print("Deduplicated modified PeaksDB peptides in 106:", len(pep106mod))
print("Total stripped PeaksDB peptides in 106:", len(pep106dup))
print("Deduplicated stripped PeaksDB peptides in 106:", len(pep106))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep106.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/106_CV_T00_GF_PDB/106B_CV_T00_GF_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/106_CV_T00_GF_PDB/106B_CV_T00_GF_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/106_CV_T00_GF_PDB/106B_CV_T00_GF_PDB_stripped_peptides.fas

# look at the stripped peptides
pep106.head()

Total modified PeaksDB peptides in 106: 85
Deduplicated modified PeaksDB peptides in 106: 85
Total stripped PeaksDB peptides in 106: 85
Deduplicated stripped PeaksDB peptides in 106: 78


,stripped_peptide
0,SCLISGWGNTK
1,LGEHNIDVLEGNEQFINAAK
2,SSGSSYPSLLQCLK
3,IITHPNFNGNTLDNDIMLIK
6,LGEHNIDVLEGNEQ


## NAAF correction and exporting files with AA and PTM totals:

In [9]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb106['A'].sum(),
        'C': peaksdb106['C'].sum(),
        'D': peaksdb106['D'].sum(),
        'E': peaksdb106['E'].sum(),
        'F': peaksdb106['F'].sum(),
        'G': peaksdb106['G'].sum(),
        'H': peaksdb106['H'].sum(),
        'I': peaksdb106['I'].sum(),
        'K': peaksdb106['K'].sum(),
        'L': peaksdb106['L'].sum(),
        'M': peaksdb106['M'].sum(),
        'N': peaksdb106['N'].sum(),
        'P': peaksdb106['P'].sum(),
        'Q': peaksdb106['Q'].sum(),
        'R': peaksdb106['R'].sum(),
        'S': peaksdb106['S'].sum(),
        'T': peaksdb106['T'].sum(),
        'V': peaksdb106['V'].sum(),
        'W': peaksdb106['W'].sum(),
        'Y': peaksdb106['Y'].sum(),
        'c-carb': peaksdb106['c-carb'].sum(),
        'm-oxid': peaksdb106['m-oxid'].sum(),
        'n-deam': peaksdb106['n-deam'].sum(),
        'q-deam': peaksdb106['q-deam'].sum(),
        'Total area': peaksdb106['Area'].sum(),
        'Total length': peaksdb106['stripped_length'].sum()
       }

totalpeaksdb106 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb106['% C w/ carb'] = totalpeaksdb106['c-carb'] / totalpeaksdb106['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb106['% M w/ oxid'] = totalpeaksdb106['m-oxid'] / totalpeaksdb106['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb106['% N w/ deam'] = totalpeaksdb106['n-deam'] / totalpeaksdb106['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb106['% Q w/ deam'] = totalpeaksdb106['q-deam'] / totalpeaksdb106['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb106['NAAF denom.'] = totalpeaksdb106['Total area'] / totalpeaksdb106['Total length']

# write modified dataframe to new txt file
totalpeaksdb106.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/106_CV_T00_GF_PDB/106B_CV_T00_GF_PDB_totals.csv")

totalpeaksdb106.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,63,18,22,53,24,124,30,69,48,92,...,1,12,4,6.627973e+08,926,1.0,0.2,0.155844,0.166667,715763.793348


In [10]:
# use the calculated NAAF factor (in totalpeaksdb106 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 715763.793348

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb106['NAAF factor'] = (peaksdb106['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb106_NAAF = peaksdb106[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb106_NAAF['A-NAAF20'] = peaksdb106_NAAF['A'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['C-NAAF20'] = peaksdb106_NAAF['C'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['D-NAAF20'] = peaksdb106_NAAF['D'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['E-NAAF20'] = peaksdb106_NAAF['E'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['F-NAAF20'] = peaksdb106_NAAF['F'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['G-NAAF20'] = peaksdb106_NAAF['G'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['H-NAAF20'] = peaksdb106_NAAF['H'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['I-NAAF20'] = peaksdb106_NAAF['I'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['K-NAAF20'] = peaksdb106_NAAF['K'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['L-NAAF20'] = peaksdb106_NAAF['L'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['M-NAAF20'] = peaksdb106_NAAF['M'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['N-NAAF20'] = peaksdb106_NAAF['N'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['P-NAAF20'] = peaksdb106_NAAF['P'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['Q-NAAF20'] = peaksdb106_NAAF['Q'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['R-NAAF20'] = peaksdb106_NAAF['R'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['S-NAAF20'] = peaksdb106_NAAF['S'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['T-NAAF20'] = peaksdb106_NAAF['T'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['V-NAAF20'] = peaksdb106_NAAF['V'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['W-NAAF20'] = peaksdb106_NAAF['W'] * peaksdb106['NAAF factor']
peaksdb106_NAAF['Y-NAAF20'] = peaksdb106_NAAF['Y'] * peaksdb106['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb106_NAAF['ccarb-NAAF20'] = peaksdb106_NAAF['c-carb'] * peaksdb106_NAAF['NAAF factor']
peaksdb106_NAAF['moxid-NAAF20'] = peaksdb106_NAAF['m-oxid'] * peaksdb106_NAAF['NAAF factor']
peaksdb106_NAAF['ndeam-NAAF20'] = peaksdb106_NAAF['n-deam'] * peaksdb106_NAAF['NAAF factor']
peaksdb106_NAAF['qdeam-NAAF20'] = peaksdb106_NAAF['q-deam'] * peaksdb106_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb106_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/106_CV_T00_GF_PDB/106B_CV_T00_GF_PDB_naaf.csv")

peaksdb106_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,0.654101,3,2,0,1,0,3,0,1,...,0.0,1.308202,1.308202,0.000000,0.654101,0.000000,1.308202,0.000000,0.0,0.000000
1,LGEHNIDVLEGNEQFINAAK,0.552557,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.552557,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,SSGSSYPSLLQCLK,0.103785,0,1,0,0,0,1,0,0,...,0.0,0.518926,0.000000,0.000000,0.000000,0.103785,0.103785,0.000000,0.0,0.000000
3,IITHPNFNGNTLDNDIMLIK,0.006895,0,0,2,0,1,1,1,4,...,0.0,0.000000,0.013789,0.000000,0.000000,0.000000,0.000000,0.006895,0.0,0.000000
4,LGEHNIDVLEGNEQFINAAK,0.005819,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.005819,0.000000,0.000000,0.000000,0.000000,0.0,0.005819


In [11]:
# made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb106 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb106_NAAF['NAAF factor'].sum(),
        'A': peaksdb106_NAAF['A-NAAF20'].sum(),
        'C': peaksdb106_NAAF['C-NAAF20'].sum(),
        'D': peaksdb106_NAAF['D-NAAF20'].sum(),
        'E': peaksdb106_NAAF['E-NAAF20'].sum(),
        'F': peaksdb106_NAAF['F-NAAF20'].sum(),
        'G': peaksdb106_NAAF['G-NAAF20'].sum(),
        'H': peaksdb106_NAAF['H-NAAF20'].sum(),
        'I': peaksdb106_NAAF['I-NAAF20'].sum(),
        'K': peaksdb106_NAAF['K-NAAF20'].sum(),
        'L': peaksdb106_NAAF['L-NAAF20'].sum(),
        'M': peaksdb106_NAAF['M-NAAF20'].sum(),
        'N': peaksdb106_NAAF['N-NAAF20'].sum(),
        'P': peaksdb106_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb106_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb106_NAAF['R-NAAF20'].sum(),
        'S': peaksdb106_NAAF['S-NAAF20'].sum(),
        'T': peaksdb106_NAAF['T-NAAF20'].sum(),
        'V': peaksdb106_NAAF['V-NAAF20'].sum(),
        'W': peaksdb106_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb106_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb106_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb106_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb106_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb106_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb106_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb106_NAAF['% C w/ carb.'] = totalpeaksdb106_NAAF['c-carb'] / totalpeaksdb106_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb106_NAAF['% M w/ oxid'] = totalpeaksdb106_NAAF['m-oxid'] / totalpeaksdb106_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb106_NAAF['% N w/ deam'] = totalpeaksdb106_NAAF['n-deam'] / totalpeaksdb106_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb106_NAAF['% Q w/ deam'] = totalpeaksdb106_NAAF['q-deam'] / totalpeaksdb106_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb106_NAAF['NAAF check'] = totalpeaksdb106_NAAF['NAAF'] / 715763.793348

# write modified dataframe to new txt file, same name + totals
totalpeaksdb106_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/106_CV_T00_GF_PDB/106B_CV_T00_GF_PDB_naaf_totals.csv")

totalpeaksdb106_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,111.06721,60.890779,2.281948,1.999403,7.70727,1.248701,7.133571,4.539808,7.977257,8.516075,...,1.570226,2.281948,0.006895,0.633386,0.1242,1.0,0.014241,0.037783,0.073996,0.000155
